In [1]:
# state transition bias and reward function for a cart pole system

import numpy as np

In [2]:
sdim = 4
adim = 1
ssize = 10
asize = 100
srange = np.array([-np.pi/4, np.pi/4, -3, 3, -2.4, 2.4, -3.5, 3.5])
arange = np.array([-10, 10])

g = 9.8
l =0.5
M = 1
m = 0.1
tau = 0.1


In [3]:
def unfold(c, d):
    return [list(np.ndindex(*d))[i] for i in c]


def foldParallel(c, d):
    p = np.array([np.prod(d[i + 1 :]) for i, _ in enumerate(d)])
    return np.sum(c * p, -1).astype(np.int32)

In [4]:
def build_spaces(srange, arange, ssize, asize, sdim, adim):
    srange = np.reshape(srange, (sdim, -1))
    arange = np.reshape(arange, (adim, -1))

    # Dicretization of [a, b] will include a, but exlude b when range if not multiple of step-size
    statespace = np.meshgrid(*[np.linspace(a, b, ssize) for (a, b) in srange])
    actionspace = np.meshgrid(*[np.linspace(a, b, asize) for (a, b) in arange])

    return statespace, actionspace

In [5]:
statespace, actionspace = build_spaces(srange, arange, ssize, asize, sdim, adim)
# print(statespace)
# cs = np.random.randint(0, statespace[0].size, samples) # current states
# ca = np.random.randint(0, actionspace[0].size, samples) # current states

# states = np.array(unfold(cs, statespace[0].shape))
# states = np.array([s[states] for s in statespace])
# actions = np.array(unfold(ca, actionspace[0].shape))
# states = np.array([a[actions] for a in actionspace])

In [63]:
def get_state_action_bias_reward(statespace, actionspace, args):
    """
    nstates: total number of states
    nactions: total number of actions
    """
    assert len(srange.shape) == 1
    #assert args.adim == 1
    
    coords = np.array(statespace).reshape(sdim, -1).transpose()
    coorda = actionspace[0]
    c = np.array(list(np.ndindex(len(coords), len(coorda))))
    cartcoord = np.c_[coords[c[:, 0]], coorda[c[:, 1]]]
    
    angaccel = (g * np.sin(cartcoord[:,0]) - (cartcoord[:, -1] + m * l * cartcoord[:,1] ** 2 * np.sin(cartcoord[:,0])) * np.cos(cartcoord[:,0]) / (M + m)) / (l * (4 / 3 - (m * np.cos(cartcoord[:,0]) ** 2) / (M+m)))
    linaccel = (cartcoord[:, -1] + m * l *(cartcoord[:,0] ** 2 * np.sin(cartcoord[:,0]) - angaccel * np.cos(cartcoord[:,0]))) / (M + m)
    
    bias = np.c_[tau*cartcoord[:,1], tau*angaccel, tau*cartcoord[:,2], tau*linaccel]
    
    reward = np.reshape(np.cos(cartcoord[:, 0]) ** 4, (len(coords), len(coorda), -1))
    
    # convert to discrete co-ordinates
    statesrange = np.reshape(srange, (sdim, -1))
    sr = np.array([np.linspace(a, b, ssize) for (a, b) in statesrange]) #use args.ssize
    bias = np.array(
        [
            np.argmin(np.abs(bias[:, i : i + 1] - sr[i : i + 1]), -1)
            for i in range(sdim)
        ]
    ).T
    bias = np.reshape(bias, (len(coords), len(coorda), -1))
    return bias, reward

In [64]:
B, R = get_state_action_bias_reward(statespace, actionspace, None)

In [22]:
B

NameError: name 'B' is not defined

In [69]:
np.unique(R)

array([0.25      , 0.45025452, 0.67468778, 0.8705127 , 0.98486545])

In [ ]:
get_reward(states,actions)

In [11]:
np.pi/4

0.7853981633974483

312.5

In [73]:
import torch

sdim = 3
params = torch.ones(2, sdim)
print("params", params)

momentum = torch.ones(2, sdim)
print("momentum",momentum)

params tensor([[1., 1., 1.],
        [1., 1., 1.]])
momentum tensor([[1., 1., 1.],
        [1., 1., 1.]])


In [74]:
def log_prob(omega):
    """
        Sampling from a trucated Gaussian distribution
    """
    mean = torch.cat([torch.zeros(1, sdim),torch.ones(1, sdim)],dim=0)
    cov = torch.cat([torch.eye(sdim).view(1,sdim,sdim),2*torch.eye(sdim).view(1,sdim,sdim)],dim=0)
    normal_log = torch.distributions.MultivariateNormal(mean, cov).log_prob(omega)
    
    return normal_log

print(log_prob(params))

tensor([-4.2568, -3.7965])


In [75]:
def gibbs(
    params,
    mass=None,
):

    if mass is None:
        dist = torch.distributions.Normal(
            torch.zeros_like(params), torch.ones_like(params)
        )
    else:
        if len(mass.shape) == 2:
            dist = torch.distributions.MultivariateNormal(
                torch.zeros_like(params), mass
            )
        elif len(mass.shape) == 1:
            dist = torch.distributions.Normal(torch.zeros_like(params), mass)
    return dist.sample()

print(gibbs(params))

tensor([[ 0.7161,  0.4260,  2.3050],
        [ 0.4742, -1.0164,  0.8682]])


In [76]:
def hamiltonian(
    params,
    momentum,
    log_prob_func,
    inv_mass=None,
):

    log_prob = log_prob_func(params)
    potential = -log_prob
    
    if inv_mass is None:
        kinetic = 0.5 * torch.sum(momentum ** 2, dim = -1)
        
    else:
        if len(inv_mass.shape) == 2:
            # Have not checked for parallel
            kinetic = 0.5 * torch.matmul(
                momentum.view(1, -1), torch.matmul(inv_mass, momentum.view(-1, 1))
            ).view(-1)
        else:
            kinetic = 0.5 * inv_mass * torch.sum(momentum ** 2, dim = -1)
    hamiltonian = potential + kinetic

    return hamiltonian

print(hamiltonian(params, momentum, log_prob))

tensor([5.7568, 5.2965])


In [77]:
torch.tensor([[2, 3],[2, 3]])

tensor([[2, 3],
        [2, 3]])

In [78]:
torch.sum(torch.tensor([2, 3]) **2 ,dim=-1)

tensor(13)

In [80]:
torch.dot(torch.tensor([2, 3]),torch.tensor([2, 3]))

tensor(13)